In [2]:
import os
import json
import tkinter as tk
from tkinter import filedialog
from google.cloud import vision
from google.cloud.vision_v1 import types
from google.oauth2 import service_account

# Set up the Google Cloud Vision client with your credentials
credentials = service_account.Credentials.from_service_account_file("alex/file/path.json")
client = vision.ImageAnnotatorClient(credentials=credentials)

# Function to detect handwritten text in an image
def detect_handwritten_text(image_path):
    # Load the image file into memory
    with open(image_path, 'rb') as image_file:
        content = image_file.read()

    # Create an image object for the Vision API
    image = types.Image(content=content)

    # Call the Vision API to detect text in the image (specifically for handwritten text)
    response = client.document_text_detection(image=image)

    # Extract and return detected text
    if response.text_annotations:
        return response.text_annotations[0].description  # Return the full detected text

    if response.error.message:
        raise Exception(f'{response.error.message}')
    
    return "No handwritten text found."

# Function to save the extracted text as JSON
def save_as_json(extracted_text, file_path):
    # Prepare the JSON data
    json_data = {
        "file_name": os.path.basename(file_path),
        "extracted_text": extracted_text
    }

    # Ask the user for the directory and file name to save the JSON
    output_path = filedialog.asksaveasfilename(defaultextension=".json", 
                                               filetypes=[("JSON files", "*.json")],
                                               title="Save as JSON")
    if output_path:
        with open(output_path, 'w') as json_file:
            json.dump(json_data, json_file, indent=4)
        result_label.config(text=f"JSON saved successfully: {output_path}")
    else:
        result_label.config(text="Save operation canceled.")

# Function to open file dialog and select an image
def select_image():
    file_path = filedialog.askopenfilename(title="Select Handwritten Note Image", 
                                           filetypes=[("Image Files", "*.jpg *.jpeg *.png *.bmp")])
    if file_path:
        # Call the text detection function
        try:
            extracted_text = detect_handwritten_text(file_path)
            result_label.config(text=f"Extracted Text:\n{extracted_text}")

            # Save the extracted text to a JSON file
            save_as_json(extracted_text, file_path)
        except Exception as e:
            result_label.config(text=f"Error: {str(e)}")
    else:
        result_label.config(text="No file selected.")

# Create the tkinter window
root = tk.Tk()
root.title("Handwritten Note Extractor (Google Vision)")

# Create the buttons and labels
select_button = tk.Button(root, text="Select Handwritten Note Image", command=select_image)
select_button.pack(pady=20)

result_label = tk.Label(root, text="", wraplength=400, justify="left")
result_label.pack(pady=20)

# Run the tkinter event loop
root.mainloop()


OSError: [Errno 22] Invalid argument: 'https://vision.googleapis.com/v1/images:annotate.json'